<a href="https://colab.research.google.com/github/raghavadevarajeurs/DeepLearning/blob/main/Assignment5/NYC_Xgboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [2]:
!wget https://raw.githubusercontent.com/raghavadevarajeurs/DeepLearning/main/Assignment5/Data/train.csv

--2021-04-19 06:11:20--  https://raw.githubusercontent.com/raghavadevarajeurs/DeepLearning/main/Assignment5/Data/train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5424127 (5.2M) [text/plain]
Saving to: ‘train.csv’

train.csv           100%[===================>]   5.17M  28.1MB/s    in 0.2s    

2021-04-19 06:11:20 (28.1 MB/s) - ‘train.csv’ saved [5424127/5424127]



In [7]:
from geopy.distance import geodesic
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [3]:
df = pd.read_csv('/content/train.csv')
df

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1
...,...,...,...,...,...,...,...,...
49995,2013-06-12 23:25:15.0000004,15.0,2013-06-12 23:25:15 UTC,-73.999973,40.748531,-74.016899,40.705993,1
49996,2015-06-22 17:19:18.0000007,7.5,2015-06-22 17:19:18 UTC,-73.984756,40.768211,-73.987366,40.760597,1
49997,2011-01-30 04:53:00.00000063,6.9,2011-01-30 04:53:00 UTC,-74.002698,40.739428,-73.998108,40.759483,1
49998,2012-11-06 07:09:00.00000069,4.5,2012-11-06 07:09:00 UTC,-73.946062,40.777567,-73.953450,40.779687,2


In [4]:
df['pickup_datetime'] =  pd.to_datetime(df['pickup_datetime'], format= "%Y-%m-%d %H:%M:%S UTC")
df['trip_start_month'] = df['pickup_datetime'].dt.month
df['trip_start_hour'] = df['pickup_datetime'].dt.hour
df['trip_start_day'] = df['pickup_datetime'].dt.day
df = df.rename(columns={'fare_amount': 'fare'})

In [5]:
df = df[(df['pickup_longitude'] <= -73.8)]
df = df[(df['pickup_latitude'] < 40.9)]
df = df[(df['dropoff_longitude'] <= -73.8)]
df = df[(df['dropoff_latitude'] < 40.9)]

df = df[(df['dropoff_longitude'] > -74.2)]
df = df[(df['pickup_longitude'] > -74.2)]
df = df[(df['pickup_latitude'] > 40.5)]
df = df[(df['dropoff_latitude'] > 40.5)]

In [8]:
df_features = df.copy()
df_features['pickup_datetime'] = pd.to_datetime(df_features['pickup_datetime'])
df_features['month'] = df_features['pickup_datetime'].dt.month
df_features['day'] = df_features['pickup_datetime'].dt.day
df_features['distance'] = df_features.apply(lambda x: geodesic((x['pickup_latitude'], x['pickup_longitude']), (x['dropoff_latitude'], x['dropoff_longitude'])).miles, axis=1)
df_features

,key,fare,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,trip_start_month,trip_start_hour,trip_start_day,month,day,distance
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21,-73.844311,40.721319,-73.841610,40.712278,1,6,17,15,6,15,0.639764
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16,-74.016048,40.711303,-73.979268,40.782004,1,1,16,5,1,5,5.246511
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00,-73.982738,40.761270,-73.991242,40.750562,2,8,0,18,8,18,0.863167
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42,-73.987130,40.733143,-73.991567,40.758092,1,4,4,21,4,21,1.737223
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00,-73.968095,40.768008,-73.956655,40.783762,1,3,7,9,3,9,1.241710
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,2013-06-12 23:25:15.0000004,15.0,2013-06-12 23:25:15,-73.999973,40.748531,-74.016899,40.705993,1,6,23,12,6,12,3.066758
49996,2015-06-22 17:19:18.0000007,7.5,2015-06-22 17:19:18,-73.984756,40.768211,-73.987366,40.760597,1,6,17,22,6,22,0.542939
49997,2011-01-30 04:53:00.00000063,6.9,2011-01-30 04:53:00,-74.002698,40.739428,-73.998108,40.759483,1,1,4,30,1,30,1.404655
49998,2012-11-06 07:09:00.00000069,4.5,2012-11-06 07:09:00,-73.946062,40.777567,-73.953450,40.779687,2,11,7,6,11,6,0.414219


In [9]:
label_col = 'fare'
X = df_features[['passenger_count', 'distance', 'month', 'day']].to_numpy()
y = df_features[[label_col]].to_numpy()

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [10]:
regressor = xgb.XGBRegressor(
    n_estimators=100,
    reg_lambda=1,
    gamma=0,
    max_depth=3
)

regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)
mean_squared_error(y_test, y_pred)

[06:13:13] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


19.37018510211464

In [12]:
min(df.pickup_longitude.min(),df.dropoff_longitude.min()), \
max(df.pickup_longitude.max(),df.dropoff_longitude.max())

(-74.187173, -73.800102)

In [13]:
min(df.pickup_latitude.min(),df.dropoff_latitude.min()), \
max(df.pickup_latitude.max(),df.dropoff_latitude.max())

(40.522263, 40.899325)

In [14]:
def select_within_test_boundary(df, BB):
    return (df.pickup_longitude >= BB[0]) & (df.pickup_longitude <= BB[1]) & \
           (df.pickup_latitude >= BB[2]) & (df.pickup_latitude <= BB[3]) & \
           (df.dropoff_longitude >= BB[0]) & (df.dropoff_longitude <= BB[1]) & \
           (df.dropoff_latitude >= BB[2]) & (df.dropoff_latitude <= BB[3])

In [15]:
BB = (-74.18, -73.80, 40.52, 40.89)
print('Old size: %d' % len(df))
train = df[select_within_test_boundary(df, BB)]
print('New size: %d' % len(train))

Old size: 47752
New size: 47720


In [20]:
train = train.drop(columns= ['key','pickup_datetime'], axis= 1).copy()

In [21]:
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(train.drop('fare', axis=1),
                                                    train['fare'], test_size=0.2, random_state = 42)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(38176, 8)
(9544, 8)
(38176,)
(9544,)


In [22]:
params = {
    'max_depth': 7,
    'gamma' :0,
    'eta':.03, 
    'subsample': 1,
    'colsample_bytree': 0.9, 
    'objective':'reg:linear',
    'eval_metric':'rmse',
    'silent': 0
}

In [23]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38176 entries, 20816 to 16561
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   pickup_longitude   38176 non-null  float64
 1   pickup_latitude    38176 non-null  float64
 2   dropoff_longitude  38176 non-null  float64
 3   dropoff_latitude   38176 non-null  float64
 4   passenger_count    38176 non-null  int64  
 5   trip_start_month   38176 non-null  int64  
 6   trip_start_hour    38176 non-null  int64  
 7   trip_start_day     38176 non-null  int64  
dtypes: float64(4), int64(4)
memory usage: 2.6 MB


In [27]:
def XGBmodel(X_train,X_test,y_train,y_test,params):
    matrix_train = xgb.DMatrix(X_train,label=y_train)
    matrix_test = xgb.DMatrix(X_test,label=y_test)
    model=xgb.train(params=params,
                    dtrain=matrix_train,num_boost_round=5000, 
                    early_stopping_rounds=10,evals=[(matrix_test,'test')])
    return model

model = XGBmodel(X_train,X_test,y_train,y_test,params)

[06:24:48] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	test-rmse:12.175
Will train until test-rmse hasn't improved in 10 rounds.
[1]	test-rmse:11.8703
[2]	test-rmse:11.5945
[3]	test-rmse:11.3078
[4]	test-rmse:11.051
[5]	test-rmse:10.7839
[6]	test-rmse:10.5458
[7]	test-rmse:10.2994
[8]	test-rmse:10.0577
[9]	test-rmse:9.82595
[10]	test-rmse:9.62152
[11]	test-rmse:9.40829
[12]	test-rmse:9.20013
[13]	test-rmse:8.99752
[14]	test-rmse:8.80065
[15]	test-rmse:8.61876
[16]	test-rmse:8.44057
[17]	test-rmse:8.26563
[18]	test-rmse:8.0956
[19]	test-rmse:7.93695
[20]	test-rmse:7.78521
[21]	test-rmse:7.63936
[22]	test-rmse:7.49894
[23]	test-rmse:7.36561
[24]	test-rmse:7.2365
[25]	test-rmse:7.11794
[26]	test-rmse:6.99162
[27]	test-rmse:6.88281
[28]	test-rmse:6.77905
[29]	test-rmse:6.66643
[30]	test-rmse:6.57196
[31]	test-rmse:6.46555
[32]	test-rmse:6.36745
[33]	test-rmse:6.27216
[34]	test-rmse:6.19136
[35]	test-rmse:6.10629
[3